<a href="https://colab.research.google.com/github/victorenriquenr/Scraping-Propiedades--Santiago-de-Chile/blob/main/ScrapeDepto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://chilepropiedades.cl/"><img src="https://miro.medium.com/v2/resize:fit:720/format:webp/0*XMbwmj-4r80bBuIg.jpg" width="400"> </a>



# **Explorando el Mercado Inmobiliario de Santiago de Chile: Extracción de Datos**

#### @uthors: Víctor E. Núñez, Julio Torres, Sanber Vizcaya.

#### Date: February 29, 2024


---
---




En la era digital actual, la capacidad de obtener información valiosa de forma rápida y eficiente es fundamental para empresas y particulares por igual. En el contexto del mercado inmobiliario de Santiago de Chile, la plataforma https://chilepropiedades.cl emerge como una fuente rica en datos sobre propiedades disponibles para compra o alquiler en la ciudad. Sin embargo, navegar por esta vasta cantidad de información de manera manual puede resultar abrumador y consumir mucho tiempo. Es aquí donde entra en juego el web scraping, una técnica automatizada que permite extraer datos de sitios web de manera rápida y eficiente, simplificando el proceso de análisis de datos.


### **EL Código**

Para este proyecto,  utilizando Python y bibliotecas como BeautifulSoup y requests, desarrollamos un script que nos permite extraer información detallada de la plataforma de propiedades mencionada. Una versión general del flujo de trabajo del script usado para este proyecto:

1. **Configuración de Encabezados y Clases**: Se definen los encabezados de la solicitud HTTP para simular un navegador y se crean las clases necesarias para estructurar los datos extraídos.

2. **Funciones de Web Scraping**: Se definen funciones para obtener el contenido HTML de una URL, buscar enlaces internos en la página y extraer datos específicos de cada propiedad inmobiliaria.

3. **Iteración a través de Páginas y Propiedades**: Se realiza un bucle para iterar a través de un número determinado de páginas en el sitio web, extrayendo datos de cada propiedad inmobiliaria en cada página.

4. **Extracción de Datos y Estructuración**: Para cada propiedad, se extraen datos como dirección, precio, número de habitaciones, etc. Estos datos se estructuran en objetos de clase y se agregan a un DataFrame de Pandas.

5. **Almacenamiento y Guardado**: Los datos extraídos se almacenan en un DataFrame de Pandas y finalmente se guardan en un archivo CSV para su posterior análisis.




## **Librerias**

In [3]:
from bs4 import BeautifulSoup
import requests


from lxml import html
import numpy as np, pandas as pd
import re
import time

## **Scraping**

In [4]:
# Establecer el encabezado para simular un navegador web
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
}

In [9]:
class Content:
  def __init__(self,direccion, precio_CLP, precio_UF, gastos_comunes, habitaciones,
               baños, piso, estacionamiento, area_total, area_util,interior,
               exterior, servicios, piso_tipo, descripcion, servicios_cercanos,
               disponible_para, fecha_publicacion, corredora, ID_publicacion,
               año_construccion, Link):
    #Inicialización de la clase con los atributos de la propiedad
    self.direccion =  direccion
    self.Link = Link
    self.precio_CLP = precio_CLP
    self.precio_UF = precio_UF
    self.gastos_comunes =  gastos_comunes
    self.habitaciones = habitaciones
    self.baños =  baños
    self.piso =  piso
    self.estacionamiento = estacionamiento
    self.area_total = area_total
    self.area_util =  area_util
    self.interior =  interior
    self.exterior = exterior
    self.servicios =  servicios
    self.piso_tipo =  piso_tipo
    self.descripcion =  descripcion
    self.servicios_cercanos =  servicios_cercanos
    self.disponible_para =  disponible_para
    self.fecha_publicacion =  fecha_publicacion
    self.corredora =  corredora
    self.año_construccion = año_construccion
    self.ID_publicacion =  ID_publicacion


  def to_dataframe(self):
    #Convertir los atributos de la propiedad en un DataFrame de Pandas
    data = {
        'direccion': self.direccion,
        'precio_CLP': self.precio_CLP,
        'precio_UF': self.precio_UF,
        'gastos_comunes': self.gastos_comunes,
        'habitaciones': self.habitaciones,
        'baños': self.baños,
        'piso': self.piso,
        'estacionamiento': self.estacionamiento,
        'area_total': self.area_total,
        'area_util': self.area_util,
        'interior': self.interior,
        'exterior': self.exterior,
        'servicios': self.servicios,
        'piso_tipo': self.piso_tipo,
        'descripcion': self.descripcion,
        'servicios_cercanos': self.servicios_cercanos,
        'disponible_para': self.disponible_para,
        'fecha_publicacion': self.fecha_publicacion,
        'corredora': self.corredora,
        'ID_publicacion': self.ID_publicacion,
        'año_construccion': self.año_construccion,
        'Link': self.Link
        }
    df = pd.DataFrame(data)
    return df

In [10]:
# Obtener el objeto BeautifulSoup y el parser de HTML de una URL dada
def getSoup(url):
  req = requests.get(url, headers = headers)
  soup = BeautifulSoup(req.text, 'html.parser')
  parser = html.fromstring(req.content)
  return soup, parser

#Extraer enlaces internos de una página web dada
def InternalLinks(url):
  internalLinks = []
  soup, parser = getSoup(url)
  links =  soup.find_all('a', href = re.compile(r'/ver-publicacion/venta/.*', re.IGNORECASE))
  for link in links:
    link = 'https://chilepropiedades.cl'+link.get('href')
    if link not in internalLinks:
      internalLinks.append(link)
  return internalLinks


In [11]:
def ScrapePage(url,numPage,delay):
    #Ajustando el límite de extracción de páginas.
    soup, parser =  getSoup(url)
    total_pages = int(soup.find_all('a', class_= "page-link")[-1].get('href').split('/')[-1])

    if numPage > total_pages:
        numPage = total_pages

    print('Extrayendo datos de un total de {} páginas de departamentos en {}:'.format(numPage, url.split('/')[-2]) )
    print(' ')

    direccion = []; Link = [];
    precio_CLP = []; precio_UF = [];
    gastos_comunes = []; habitaciones = [];
    baños = []; piso = [];
    estacionamiento = [];
    area_total = []; area_util = [];
    disponible_para = [];
    fecha_publicacion = []; ID_publicacion = [];
    interior = []; exterior=[];
    piso_tipo = []; servicios = [];
    descripcion = []; servicios_cercanos = [];
    corredora = []; año_construccion = [];
    url_ = url
    while numPage != 0:

        links =  InternalLinks(url_)
        for element in links:
            soup,parser = getSoup(element)

            #Link------------------------------------------------------
            Link.append(element)
            #direccion-------------------------------------------------
            try:
                direccion.append(soup.find('h1').get_text().replace('\n', '').strip())
            except:
                direccion.append(np.nan)

            #-----------------------------------------------------------
            #-----------------------------------------------------------

            dict={}
            for key,value in zip(soup.find_all('div', class_ = "clp-description-label col-6"),
                                 soup.find_all('div', class_ = "clp-description-value col-6")
                                ):
                key =  key.get_text().strip()
                value =  value.get_text().replace('  ','').strip()
                dict[key] =  value

            #-----------------------------------------------------------
           #Precio
            try:
                if dict['Valor (CLP aprox.)*:']:
                    Valor_CLP_ = dict['Valor (CLP aprox.)*:'].split()[1]
                    precio_CLP.append(Valor_CLP_)
                    Valor_UF_  = dict['Valor:'].split()[1]
                    precio_UF.append(Valor_UF_)
            except:
                Valor_CLP_  = dict['Valor:'].split()[1]
                precio_CLP.append(Valor_CLP_)
                Valor_UF_ = dict['Valor (UF aprox.)*:'].split()[1]
                precio_UF.append(Valor_UF_)


            #Gastos Comunes--------------------------------------------
            try:
                gastos_comunes.append(dict['Gastos Comunes:'].split()[1])
            except:
                gastos_comunes.append(np.nan)
            #Habitaciones----------------------------------------------
            try:
                habitaciones.append(int(dict['Habitaciones:']))
            except:
                habitaciones.append(np.nan)
            #Baños-----------------------------------------------------
            try:
                baños.append(int(dict['Baño:']))
            except:
                baños.append(np.nan)
            #Piso--------------------------------------------------------
            try:
                piso.append(int(dict['Piso:']))
            except:
                piso.append(np.nan)
            #Estacionamiento------------------------------------------
            try:
                estacionamiento.append(int(dict['Estacionamientos:']))
            except:
                estacionamiento.append('No')
            #Area Total-------------------------------------------------
            try:
                area_total.append(dict['Superficie Total:'])
            except:
                area_total.append(np.nan)
            #Area Util-------------------------------------------
            try:
                area_util.append(dict['Superficie Útil:'])
            except:
                area_util.append(np.nan)
            #Año de construcción:
            try:
                año_construccion.append(dict['Año Construcción:'])
            except:
                año_construccion.append(np.nan)


            #----------------------------------------------------------
            #----------------------------------------------------------

            dict_2 = {}

            for key,value in zip(soup.find_all('div', class_ = "col-6 clp-description-label"),
                                 soup.find_all('div', class_ = "col-6 clp-description-value")
                                ):
                key =  key.get_text().strip()
                value =  value.get_text().replace('  ','').strip()
                dict_2[key] =  value
            #----------------------------------------------------------
            #Disponible para--------------------------------------------------
            try:
                disponible_para.append(dict_2['Tipo de propiedad:'])
            except:
                disponible_para.append(np.nan)
            #Fecha de Publicación------------------------------------------
            try:
                fecha_publicacion.append(dict_2['Fecha Publicación:'])
            except:
                fecha_publicacion.append(np.nan)
            #ID--------------------------------------------------------
            try:
                ID_publicacion.append(dict_2['Código aviso:'])
            except:
                ID_publicacion.append(np.nan)
            #----------------------------------------------------------
            # Servicios
            #----------------------------------------------------------
            serv_ = {}
            if soup.find('ul', class_="clp-equipment-list"):
                for element in soup.find('ul', class_="clp-equipment-list").get_text().strip().split('\n')[:4]:
                    try:
                        key = element.split(':')[0]
                        value = element.split(':')[1]
                        serv_[key] = value
                    except:
                        None

            else:
                serv_['Interior'] = np.nan
                serv_['Exterior'] = np.nan
                serv_['Servicios'] = np.nan
                serv_['Piso'] = np.nan

            #Interior--------------------------------------------------------
            try:
                interior.append(serv_['Interior'])
            except:
                interior.append(np.nan)
            #Exterior--------------------------------------------------------
            try:
                exterior.append(serv_['Exterior'])
            except:
                exterior.append(np.nan)
            #Servicios--------------------------------------------------------
            try:
                servicios.append(serv_['Servicios'])
            except:
                servicios.append(np.nan)
            #Tipo de Piso---------------------------------------------------
            try:
                piso_tipo.append(serv_['Piso'])
            except:
                piso_tipo.append(np.nan)

            #Descripción-----------------------------------------------------
            text =[]
            if soup.find('div', class_="clp-description-box"):
                try:
                    for element in soup.find('div', class_="clp-description-box"):
                        text.append(element.get_text().strip())
                    descripcion.append(" ".join(text))
                except:
                    descripcion.append(np.nan)
            else:
                descripcion.append(np.nan)
            #Servicios Cercanos ---------------------------------------------
            if soup.find_all('h2', string = 'Comodidades y Lugares de Interés'):
                try:
                    list_ = []
                    for div in soup.find_all('div', class_= 'amenity-text'):
                        Nearby_amenities_ = div.find('p').get_text().replace('', '').split('\n')
                        Nearby_amenities_ = list(filter(lambda x: x.strip(), Nearby_amenities_))
                        list_.append(Nearby_amenities_[0].strip()+' ('+Nearby_amenities_[-1].strip()+')')
                    servicios_cercanos.append(list_)
                except:
                    servicios_cercanos.append(np.nan)
            else:
                servicios_cercanos.append(np.nan)
            #Corredora-----------------------------------
            if soup.find_all('h2', string='Corredora'):
                try:
                    Real_estate_agent_ = soup.find('div', class_= "col-sm-8 clp-user-contact-details-table").find('a').get_text()
                    corredora.append(Real_estate_agent_)
                except:
                    corredora.append(np.nan)
            else:
                corredora.append(np.nan)


        time.sleep(delay)
        numPage = numPage-1
        if numPage == 0:
            break
        next_soup, next_ṕarser = getSoup(url_)
        url_ = next_soup.find('link', {'rel':'next'}).get('href')

    return Content(direccion, precio_CLP, precio_UF, gastos_comunes, habitaciones,
               baños, piso, estacionamiento, area_total, area_util,interior,
               exterior, servicios, piso_tipo, descripcion, servicios_cercanos,
               disponible_para, fecha_publicacion, corredora, ID_publicacion,
               año_construccion, Link)


# Creando DataFrames vacíos.
dataframe1 = pd.DataFrame()
dataframe2 = pd.DataFrame()
dataframe3 = pd.DataFrame()
dataframe4 = pd.DataFrame()
dataframe5 = pd.DataFrame()
dataframe6 = pd.DataFrame()
dataframe7 = pd.DataFrame()
dataframe8 = pd.DataFrame()

for element in ['santiago','las-condes', 'nunoa', 'providencia', 'estacion-central',
               'san-miguel', 'vitacura', 'independencia']:
    start_url = 'https://chilepropiedades.cl/propiedades/venta/departamento/{}/0'.format(element)

    #Iniciamos extracción:
    content = ScrapePage(start_url,500,1)

    new_df = content.to_dataframe()  # Llama al método to_dataframe() del objeto Content.

    # Determina qué DataFrame corresponde y concatena el nuevo DataFrame.
    if element == 'santiago':
        dataframe1 = pd.concat([dataframe1, new_df],axis = 0, ignore_index=True)
    elif element == 'las-condes':
        dataframe2 = pd.concat([dataframe2, new_df], ignore_index=True)
    elif element == 'nunoa':
        dataframe3 = pd.concat([dataframe3, new_df], ignore_index=True)
    elif element == 'providencia':
        dataframe4 = pd.concat([dataframe4, new_df], ignore_index=True)
    elif element == 'estacion-central':
        dataframe5 = pd.concat([dataframe5, new_df], ignore_index=True)
    elif element == 'san-miguel':
        dataframe6 = pd.concat([dataframe6, new_df], ignore_index=True)
    elif element == 'vitacura':
        dataframe7 = pd.concat([dataframe7, new_df], ignore_index=True)
    elif element == 'independencia':
        dataframe8 = pd.concat([dataframe8, new_df], ignore_index=True)


final_df = pd.concat([dataframe1, dataframe2, dataframe3, dataframe4,
                     dataframe5, dataframe6, dataframe7, dataframe8], axis = 0).reset_index(drop =  True)
#Guardamos
final_df.to_csv('./Dataset_DeptoStgoCHL.csv')

print('¡Extracción completada!')

Extrayendo datos de un total de 1 páginas de departamentos en santiago:
 
Extrayendo datos de un total de 1 páginas de departamentos en las-condes:
 
Extrayendo datos de un total de 1 páginas de departamentos en nunoa:
 
Extrayendo datos de un total de 1 páginas de departamentos en providencia:
 
Extrayendo datos de un total de 1 páginas de departamentos en estacion-central:
 
Extrayendo datos de un total de 1 páginas de departamentos en san-miguel:
 
Extrayendo datos de un total de 1 páginas de departamentos en vitacura:
 
Extrayendo datos de un total de 1 páginas de departamentos en independencia:
 
¡Extracción completada!
